In [1]:
import json
import sys

def fix_notebook_metadata(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            notebook = json.load(f)

        # Remove widgets metadata entirely if it exists
        if 'widgets' in notebook.get('metadata', {}):
            del notebook['metadata']['widgets']
            print(f"Removed 'widgets' from metadata in {file_path}")
        else:
            print(f"No 'widgets' found in metadata for {file_path}")

        # Also check for widget state in individual cells
        for i, cell in enumerate(notebook.get('cells', [])):
            if 'widgets' in cell.get('metadata', {}):
                del cell['metadata']['widgets']
                print(f"Removed 'widgets' from metadata in cell {i}")

        # Write the fixed notebook back to file
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(notebook, f, indent=2)
            
        print(f"Successfully updated notebook: {file_path}")
        return True
    except Exception as e:
        print(f"Error fixing notebook {file_path}: {str(e)}")
        return False

if __name__ == "__main__":
    if len(sys.argv) > 1:
        notebook_path = sys.argv[1]
    else:
        notebook_path = input("Enter the path to your notebook: ")
    
    fix_notebook_metadata(notebook_path)

Error fixing notebook --f=c:\Users\jisna\AppData\Roaming\jupyter\runtime\kernel-v33c88c0b8f0a227c66974f7ad2f7d4438bfe83fbe.json: [Errno 22] Invalid argument: '--f=c:\\Users\\jisna\\AppData\\Roaming\\jupyter\\runtime\\kernel-v33c88c0b8f0a227c66974f7ad2f7d4438bfe83fbe.json'


In [1]:
import json

def fix_notebook_metadata(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        notebook = json.load(f)

    if 'widgets' in notebook.get('metadata', {}):
        del notebook['metadata']['widgets']

    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(notebook, f, indent=2)

# Replace with the path to your notebook
fix_notebook_metadata("Copy_of_Agentic_AI_powered_chatbot_CLAT_2025.ipynb")

In [1]:
import llama_index

In [2]:
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
OPENAI_KEY = os.getenv("OPENAI_KEY")

In [5]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", api_key=OPENAI_KEY)

In [11]:
TVLY_API = os.getenv("TVLY_API")

In [12]:
import os
import asyncio
from tavily import AsyncTavilyClient
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.workflow import FunctionAgent, AgentWorkflow
from llama_index.core.memory import ChatMemoryBuffer





if not TVLY_API:
    raise ValueError("Tavily API key not found. Please set the TVLY_API environment variable.")
if not OPENAI_KEY:
    raise ValueError("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")

async def search_web(query: str) -> str:
    """Search the web for information and return results with limited content."""
    try:
        client = AsyncTavilyClient(api_key=TVLY_API)
        results = await client.search(query, include_answer=True, include_raw_content=False)


        search_result = results.get('answer', '') + "\n\nSources:\n"


        for i, result in enumerate(results.get('results', [])[:3]):
            search_result += f"- {result.get('title')}: {result.get('url')}\n"
            # Limit content to ~200 chars per source
            if 'content' in result:
                search_result += f"  {result.get('content')[:200]}...\n"

        return search_result
    except Exception as e:
        return f"Error searching the web: {str(e)}"




In [13]:
llm = OpenAI(
    model="gpt-3.5-turbo",
    api_key=OPENAI_KEY,
    max_tokens=4096,  # Limit output tokens
    temperature=0.7
)

# Create the search tool
tavily_search_tool = FunctionTool.from_defaults(
    fn=search_web,
    name="web_search",
    description="Use this tool to search the web for recent information. Always use this tool when asked about current events, dates, or specific facts."
)
system_prompt_web= (
    "You are a helpful assistant that uses web search to find the most recent and relevant information about CLAT 2025."
     "Use the WebSearch tool to find answers that are not available in official documents — such as current news, announcements, coaching advice, student experiences, preparation tips, or anything dynamic or updated online."
    "If the query relates to official syllabus, exam dates, eligibility, or brochure info, redirect the query to the agent handling official documents instead."
)



# Create memory
memory = ChatMemoryBuffer.from_defaults(token_limit=4000)

# Create agent
web_agent = FunctionAgent(
    name="websearch_agent",
    description = "searches web to answer queries with accurate up-to-date answers",
    tools=[tavily_search_tool],
    llm=llm,
    system_prompt=system_prompt_web,
    memory=memory,
    verbose=False
)



In [14]:
from llama_index.readers.web import AgentQLWebReader
API_AGENTQL=os.getenv("API_AGENTQL")
agentql_reader = AgentQLWebReader(
    api_key= API_AGENTQL,  # Replace with key from https://dev.agentql.com
    params={
        "is_scroll_to_bottom_enabled": True,

    }
)
clat_url = "https://consortiumofnlus.ac.in/clat-2025/"


clat_document = agentql_reader.load_data(
    url=clat_url,
    prompt="Extract all text visible on the page, including syllabus details, important announcements, and content inside dropdown menus or expandable sections."
)



In [15]:
from llama_index.core import VectorStoreIndex
#from llama_index.storage.storage_context import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import chromadb
from chromadb import PersistentClient

chroma_client = PersistentClient(path="./chroma_db")
chroma_collection = chroma_client.get_or_create_collection("clat_2025_collection")


vector_store = ChromaVectorStore(chroma_collection=chroma_collection)


c:\Users\jisna\Agentic_AI_Powered_Chatbot_for_Queries_Related_to_CLAT_2025\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.tools import QueryEngineTool
from llama_index.core.agent.workflow import ReActAgent
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

HF_TOKEN = os.getenv("HF_TOKEN")
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)


pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=100, chunk_overlap=20),

        Settings.embed_model, 
    ],
    vector_store=vector_store,
)


pipeline.run(documents=clat_document)

# Create your index
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(vector_store)
# Save the index for future use
llm = OpenAI(
    model="gpt-3.5-turbo",
    api_key=OPENAI_KEY,
    max_tokens=4096,  # Limit output tokens
    temperature=0.7
)
index.storage_context.persist("clat_2025_index")
query_engine = index.as_query_engine(llm=llm)
query_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="CLAT2025OfficialDocs",
    description="Best used when the query relates to information from the official CLAT 2025 website, and when semantic search provides a confident, relevant match from the official documents."
)


In [19]:
system_prompt_query=(
    "You are an expert assistant with access to official CLAT 2025 documentation."
    "Use the CLAT2025OfficialDocs tool to answer questions about the official syllabus, exam pattern, eligibility criteria, important dates, fees, and other information published on the official CLAT website or brochures"
    "Do not attempt to answer questions that require up-to-date news, coaching advice, or discussion-based content — those should be handled by the web search agent."

)
query_agent=ReActAgent(
    name="Rag_Agent",
    description="Uses vector database for answering queries related to CLAT2025 where semantic search of the vector database gives accurate answers.",
    llm=llm,
    system_prompt=system_prompt_query,
    memory=memory,
    verbose=False
)

In [29]:
# Create workflow
from llama_index.core.workflow import Context
workflow = AgentWorkflow(agents=[web_agent,query_agent], root_agent="websearch_agent")





In [21]:
import asyncio
import nest_asyncio


nest_asyncio.apply()


async def main():
    try:
        response = await workflow.run("Give me last year’s cut-off for NLSIU Bangalore.")
        print(response)
    except Exception as e:
        print(f"Error running workflow: {str(e)}")

    pass


if __name__ == "__main__":
    asyncio.run(main())

The NLSIU Bangalore last year cut-off for the General category was rank 102 in CLAT 2024. For 2025, the expected cut-off is 95+ marks. The cut-off is based on factors like exam difficulty and applicant numbers.

Sources:
- [Exam Charcha - NLSIU Bangalore Cut-Off 2025, 2024 & Past Years](https://examcharcha.in/nlsiu-bangalore-cut-off/)
- [College Dekho - NLSIU Bangalore Cut Off 2025, 2024, 2023 - Previous Year's Cut off List](https://www.collegedekho.com/colleges/nlsiu-cutoff)
- [IMS India - NLSIU Bangalore Cut off 2024: College-wise Cut offs](https://www.imsindia.com/blog/clat/nlsiu-bangalore-cut-off/)


In [22]:
import asyncio
import nest_asyncio


nest_asyncio.apply()


async def main():
    try:
        response = await workflow.run("what is the eligibility criteria for CLAT2025")
        print(response)
    except Exception as e:
        print(f"Error running workflow: {str(e)}")

    pass


if __name__ == "__main__":
    asyncio.run(main())

The eligibility criteria for CLAT 2025 typically include a minimum educational qualification of passing the 10+2 examination or equivalent with a minimum percentage requirement. Additionally, there may be age limits and nationality requirements. It's important to check the official CLAT website or notification for the specific eligibility criteria for CLAT 2025.


In [23]:
import asyncio
import nest_asyncio


nest_asyncio.apply()


async def main():
    try:
        response = await workflow.run("what is the eligibility criteria for CLAT2025 in the official website for CLAT2025")
        print(response)
    except Exception as e:
        print(f"Error running workflow: {str(e)}")

    pass


if __name__ == "__main__":
    asyncio.run(main())

The eligibility criteria for CLAT2025 can typically include educational qualifications, age limit, and other requirements set by the Consortium of National Law Universities. To get the most accurate and up-to-date information, it's best to visit the official website of CLAT2025.


In [24]:
import asyncio
import nest_asyncio


nest_asyncio.apply()


async def main():
    try:
        response = await workflow.run("What is the syllabus for CLAT2025")
        print(response)
    except Exception as e:
        print(f"Error running workflow: {str(e)}")

    pass


if __name__ == "__main__":
    asyncio.run(main())

The syllabus for CLAT 2025 typically includes subjects like English, General Knowledge and Current Affairs, Elementary Mathematics, Legal Aptitude, and Logical Reasoning. It is important to refer to the official website or brochure for the most accurate and detailed syllabus information.


In [25]:
import asyncio
import nest_asyncio


nest_asyncio.apply()


async def main():
    try:
        response = await workflow.run("when is CLAT 2025 supposed to take place")
        print(response)
    except Exception as e:
        print(f"Error running workflow: {str(e)}")

    pass


if __name__ == "__main__":
    asyncio.run(main())

The CLAT 2025 exam is scheduled to take place on December 1, 2024. Registration for the exam will begin on July 15, 2024, and the last date for registration is October 15, 2024. The admit card for the exam will be released on November 21, 2024.

Sources:
- [Law Prep Tutorial - CLAT 2025 Notification, Registration, Exam Dates](https://www.lawpreptutorial.com/blog/clat/2025-details/)
- [CollegeDunia - CLAT 2025 Exam Date (Dec 1), Application Form, Admit Card, Result](https://collegedunia.com/exams/clat/exam-dates)
- [CLAT 2025 Calendar - Consortium of NLUs](https://consortiumofnlus.ac.in/clat-2025/notifications/CLAT-2025-Calendar.pdf)


In [26]:
import asyncio
import nest_asyncio


nest_asyncio.apply()


async def main():
    try:
        response = await workflow.run("when is next CLAT exam after 2025 supposed to take place")
        print(response)
    except Exception as e:
        print(f"Error running workflow: {str(e)}")

    pass


if __name__ == "__main__":
    asyncio.run(main())

The next CLAT exam after 2025 is scheduled to take place on December 1, 2025.


In [32]:
import asyncio
import nest_asyncio


nest_asyncio.apply()


async def main():
    try:
        response = await workflow.run("what are main updates about CLAT2025 on the official website")
        print(response)
    except Exception as e:
        print(f"Error running workflow: {str(e)}")

    pass


if __name__ == "__main__":
    asyncio.run(main())

The main update about CLAT 2025 on the official website is that the application process will open soon. For the latest information, you can check the official website at clat.ac.in.


In [31]:
import asyncio
import nest_asyncio


nest_asyncio.apply()


async def main():
    try:
        response = await workflow.run("what is the syllabus for Dec CLAT 2025 on official CLAT 2025 website")
        print(response)
    except Exception as e:
        print(f"Error running workflow: {str(e)}")

    pass


if __name__ == "__main__":
    asyncio.run(main())

The CLAT 2025 UG syllabus for the December exam includes English Language, Current Affairs, Legal Reasoning, Logical Reasoning, and Quantitative Techniques. The exam is 2 hours long with 120 multiple-choice questions. Negative marking applies for wrong answers.

You can find more information on the official Consortium of NLUs website:
- [CLAT 2025 - Consortium of NLUs](https://consortiumofnlus.ac.in/clat-2025/pg-question-format.html)
- [CLAT 2025 UG Instructions - Consortium of NLUs](https://consortiumofnlus.ac.in/clat-2025/ug-instructions.html)
- [CLAT PG 2025 Syllabus - Consortium of NLUs](https://consortiumofnlus.ac.in/clat-2025/pg-syllabus.html)
